<a href="https://colab.research.google.com/github/sledhead/simple_python/blob/main/webpage_scrapv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Modules (Libraries)

*   Link to Google Drive
*   Install Modules to interact with xml
*   Link Required libraries

In [ ]:
import pandas as pd
import requests
#Will display upload box to user to allow a file to upload and encrypted.
from google.colab import drive
drive.mount('/content/drive')

!pip install xmltodict
import xmltodict

import time
import os
import shutil

# Senator Vote Records


*   Get the senator vote records for each session
*   Convert the records into individual vote tallies
*   Provide User with ability to control time period

Section below will gather xml file for each year of the senate vote.... 2017 to current

In [ ]:
#allow the user the choice to define the begin years and end years....
#https://www.senate.gov/legislative/votes_new.htm
#build a list from the site above....
#will start with 1990 and work forward to current day
#1990 = vote_menu_101_1.xml

def combine_nums(cong_id,sess_id):
  combine_str = str(cong_id) + '_' + str(sess_id)
  return combine_str

year_lst = range(1989,2025,1)
congress_id_lst = range(101,119,1)
session_lst = [1,2]

congress_session_lst = [ [combine_nums(congress_id_num, session_id) for session_id in session_lst ] for congress_id_num in congress_id_lst ]
flatten_congress_session_lst = [ final_out for pair in congress_session_lst for final_out in pair ]
#flatten_list = [j for sub in ini_list for j in sub]
#zip()

combine_congress_year = zip(year_lst,flatten_congress_session_lst)
vote_menu_dict = {}

for year_num in combine_congress_year:
  vote_menu_dict[year_num[0]] = 'vote_menu_' + year_num[1] + '.xml'


In [ ]:
# @title Please Select the Year Range  Min to Max { display-mode: "form" }

min_year = '2017' # @param ['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
max_year = '2024' # @param ['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

#build list of full file path required by the user:
#take the min and max and build a range of year required
year_num_req = range(int(min_year),(int(max_year)+1),1)
base_web_link = 'https://www.senate.gov/legislative/LIS/roll_call_lists/'

web_link_lst = []
for each_yr in year_num_req:
  file_name_part = vote_menu_dict[each_yr]
  complete_path = base_web_link + file_name_part
  web_link_lst.append(complete_path)


#now get the requested files from website....

for single_web_link in web_link_lst:
  !wget { single_web_link }

Next Section below will open xml and then use this file to build the list of file links for gather senator votes

In [ ]:
#Base file path for local folder struct in vm
current_dir = os.getcwd()
local_base_folder_path = os.path.join(current_dir,'Senator_data')
if( os.path.exists(local_base_folder_path) == False ):
  os.makedirs(local_base_folder_path)

congress_session_folder_names = []
full_http_link_file_path = []
each_vote_info = []
#vote_menu_file_lst = ['vote_menu_115_1.xml','vote_menu_115_2.xml','vote_menu_116_1.xml','vote_menu_116_2.xml','vote_menu_117_1.xml','vote_menu_117_2.xml','vote_menu_118_1.xml','vote_menu_118_2.xml']

#for each_year_call_lst in vote_menu_file_lst:
for each_year in year_num_req:
  file_name_part = vote_menu_dict[each_year]

  xml_file = open(file_name_part, 'r')
  xml_data_str = xml_file.read()

  xml_to_dict_obj = xmltodict.parse(xml_data_str)

  congress_num = xml_to_dict_obj['vote_summary']['congress']
  congress_sec_num = xml_to_dict_obj['vote_summary']['session']

  congress_lst_vote = xml_to_dict_obj['vote_summary']['votes']['vote']



  base_link_path = 'https://www.senate.gov/legislative/LIS/roll_call_votes/'
  full_base_link_path = base_link_path + 'vote' + str(congress_num) + str(congress_sec_num) + '/vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_'
  congress_session_folder_names.append('vote_' + str(congress_num) + '_' + str(congress_sec_num))
  print(full_base_link_path)

  for each_vote_section in congress_lst_vote:
    single_vote_num = each_vote_section['vote_number']
    single_vote_info_dict = {}
    #build full link path for getting xml data....
    #https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_00420.htm

    single_file_link_path = full_base_link_path + str(single_vote_num) + '.xml'
    full_http_link_file_path.append(single_file_link_path)
    single_vote_info_dict['full_http_path'] = single_file_link_path
    single_vote_info_dict['dest_folder'] = 'vote_' + str(congress_num) + '_' + str(congress_sec_num)
    single_vote_info_dict['file_name'] = 'vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_' + str(single_vote_num) + '.xml'
    each_vote_info.append(single_vote_info_dict)


#section will build the temp folder directory struct for this vm instanse
print('..............................................................')
for new_folder_name in congress_session_folder_names:
  #build new folder path and make
  local_folder_path = os.path.join(current_dir,'Senator_data',new_folder_name)
  if( os.path.exists(local_folder_path) == False ):
    print(f'The following folder was added: {local_folder_path}')
    os.makedirs(local_folder_path)

In [ ]:
# @title Please select local file storage only or local plus drive backup { display-mode: "form" }

location = 'drive_backup' # @param ['local_only','drive_backup']
backup_path = '/content/drive/MyDrive/Encryption_test/Senator_data' # @param {type:"string"}


for single_vote_xml_file in each_vote_info:
  #get the request xml data files
  #get the xml data files
  if( location == 'local_only' ):
    #just store xml file on the local vm....
    local_dest_folder_path = os.path.join(current_dir,'Senator_data',single_vote_xml_file['dest_folder'])
    !wget -P {local_dest_folder_path} {single_vote_xml_file['full_http_path']}

  if( location == 'drive_backup'):
    #more stuff to do...
    #does drive path exists???
    local_dest_folder_path = os.path.join(backup_path,single_vote_xml_file['dest_folder'])
    if( os.path.exists(local_dest_folder_path) == False ):
      os.makedirs(local_dest_folder_path)

    #before download a new copy, check and see if it exists on the drive backup...
    check_for_xml_file = os.path.join(backup_path,single_vote_xml_file['dest_folder'], single_vote_xml_file['file_name'])
    if( os.path.exists(check_for_xml_file) == False ):
      #does not exist in back_up, so download
      !wget -P {local_dest_folder_path} {single_vote_xml_file['full_http_path']}

    else:
      #file was found...
      print(f'Found:::::: {check_for_xml_file}')